In [1]:
from nltk.metrics.distance import edit_distance

In [2]:
correct_words = ['gulshan e hadeed', 'gulshan e iqbal', 'defence', 'clifton', 'meher plaza', 'al murtaza heights', 'al murtaza height']

incorrect_words= ['gulshen iqbal', 'defnse', 'klifton', 'mehar plaza', 'al murteza heights']

for word in incorrect_words:
		temp = [(edit_distance(word, w),w) for w in correct_words]
		# print(sorted(temp, key = lambda val:val[0])[0][1])

		if len(temp) > 1: 
			temp = [min(temp, key=lambda t: t[0])]

		print(temp)
		# print(temp[0][1])

# print(edit_distance(incorrect_words[2], correct_words[3]), correct_words[3])

[(3, 'gulshan e iqbal')]
[(2, 'defence')]
[(1, 'clifton')]
[(1, 'meher plaza')]
[(1, 'al murtaza heights')]


In [3]:
import pandas as pd

In [5]:
def load_corpus(file_name, pandas=False):
    if pandas==True:
        df = pd.read_csv(file_name, sep="\t", header=None, names=["Neighborhood"])
        return df
    else:
        with open(file_name, "r") as file:
            neighborhood_names = file.readlines()    
        refined_name_list = [name.strip() for name in neighborhood_names]
        return refined_name_list


def lowercase_conversion(text_str):
    return text_str.lower()


corpus_data = load_corpus("karachi_neighbourhoods.txt")

file_name = "karachi_neighbourhoods_lowered.txt"

with open(file_name, "w") as file:
    for item in corpus_data:
        item = lowercase_conversion(item)
        file.write(item + "\n")

new_corpus = load_corpus("karachi_neighbourhoods_lowered.txt", True)

print(new_corpus.head())

print(f"File '{file_name}' has been created.")



        Neighborhood
0         abbas town
1     abbasi shaheed
2  abdul rehman goth
3      abdullah goth
4           abidabad
File 'karachi_neighbourhoods_lowered.txt' has been created.


In [5]:
abbreviations = {
    'Apt': 'Apartment',
    'Bldg.': 'Building',
    'Cantt.': 'Cantonment',
    'Col.': 'Colony',
    'Ext.': 'Extension',
    'Hno.': 'House Number',
    'Mkt.': 'Market',
    'Ngr.': 'Nagar',
    'P.O.': 'Post Office',
    'P.O.B': 'Post Office Box',
    'Pl.': 'Place',
    'Sec.': 'Sector',
    'Twn.': 'Town',
    'Vlg.': 'Village',
    'Dist.': 'District',
    'Rd.': 'Road',
    'Jct.': 'Junction',
    'Ln.': 'Lane',
    'Masjid': 'Mosque',
    'Fl.': 'Floor',
    'Blk.': 'Block',
    'Sch.': 'School',
    'Univ.': 'University',
    'Corp.': 'Corporation',
    'Ctr.': 'Center',
    'Plz.': 'Plaza',
    'Ph.': 'Phase',
    'G.P.O.': 'General Post Office',
    'Bzr.': 'Bazaar',
    'Est.': 'Estate',
    'St.': 'Street',
    'Ave': 'Avenue',
    'Blvd': 'Boulevard',
    'Pkwy': 'Parkway',
    'Hwy': 'Highway',
    'Res.': 'Residential',
    'Phase': 'Phase',
    'Khy':'Khayaban'
    }

punctuation removing: (from sources)
- city areas corpus (only has dashes -) -> (solved for capitalization)
- abbriviations json -> punctuation + capitalization
- data to process -> lowered plus removing all capitalization except for , and - 



In [11]:
sample1 = 'House # C38, Block 8, Gulshan-e-Iqbal, Karachi'
sample2 = 'House No. 123, Street 5, Phase 7, DHA'
sample3 = "House No. 123, St. 5, Phase 7, DHA"

In [30]:
import re

def tokenize_address(address):
    # Word Tokenization
    tokens = address.split()

    # Regex Tokenization for capturing specific patterns
    # You can customize the regex patterns based on the address format in Pakistan
    regex_patterns = [
        r'\d+',  # Capture numerical values (e.g., house numbers)
        r'[A-Za-z]+',  # Capture alphabetic words (e.g., street names, city names)
        r'\b[A-Za-z]{2}\b'  # Capture two-letter abbreviations (e.g., state codes)
    ]
    tokens_new = []
    for pattern in regex_patterns:
        regex_tokens = re.findall(pattern, address)
        tokens_new.extend(regex_tokens)

    return tokens_new


In [33]:
address = "12, Main Street, Lahore, Punjab"
# tokens = tokenize_address(address)
# print(tokens)

print(tokenize_address(address))
print(tokenize_address(sample1))
print(tokenize_address(sample2))
print(tokenize_address(sample3))


['12', 'Main', 'Street', 'Lahore', 'Punjab']
['38', '8', 'House', 'C', 'Block', 'Gulshan', 'e', 'Iqbal', 'Karachi']
['123', '5', '7', 'House', 'No', 'Street', 'Phase', 'DHA', 'No']
['123', '5', '7', 'House', 'No', 'St', 'Phase', 'DHA', 'No', 'St']


In [1]:
def check_address_type(address):
    address = address.lower()

    house_keywords = ['house no', 'house number', 'house #', 'house']
    apartment_keywords = ['flat no', 'flat number', 'flat #', 'flat', 'apartment', 'building']

    for keyword in house_keywords:
        if keyword in address:
            return 'House'

    for keyword in apartment_keywords:
        if keyword in address:
            return 'Apartment'

    return 'Unknown'

# Example usage:
address1 = "House No. 123, Main Street, New York"
address2 = "Apartment 7B, Building X, Chicago"
address3 = "1234 Park Avenue, Los Angeles"

type1 = check_address_type(address1)
type2 = check_address_type(address2)
type3 = check_address_type(address3)

print(f"Address 1 Type: {type1}")
print(f"Address 2 Type: {type2}")
print(f"Address 3 Type: {type3}")


Address 1 Type: House
Address 2 Type: Apartment
Address 3 Type: Unknown


In [2]:
def check_address_type(address):
    address = address.lower()

    house_keywords = ['house no', 'house number', 'house #', 'house']
    apartment_keywords = ['flat no', 'flat number', 'flat #', 'flat', 'apartment', 'building']

    house_found = any(keyword in address for keyword in house_keywords)
    apartment_found = any(keyword in address for keyword in apartment_keywords)

    if house_found and apartment_found:
        return 'Both House and Apartment'
    elif house_found:
        return 'House'
    elif apartment_found:
        return 'Apartment'
    else:
        return 'Unknown'

# Example usage:
address1 = "House No. 123, Main Street, New York"
address2 = "Apartment 7B, Building X, Chicago"
address3 = "1234 Park Avenue, Los Angeles"
address4 = "Flat No. 456, ABC Apartments, London"
address5 = "House #789, XYZ Building, Sydney"

type1 = check_address_type(address1)
type2 = check_address_type(address2)
type3 = check_address_type(address3)
type4 = check_address_type(address4)
type5 = check_address_type(address5)

print(f"Address 1 Type: {type1}")
print(f"Address 2 Type: {type2}")
print(f"Address 3 Type: {type3}")
print(f"Address 4 Type: {type4}")
print(f"Address 5 Type: {type5}")


Address 1 Type: House
Address 2 Type: Apartment
Address 3 Type: Unknown
Address 4 Type: Apartment
Address 5 Type: Both House and Apartment


In [5]:
import random
import csv

house_keywords = ['house no', 'house number', 'house #', 'house']
apartment_keywords = ['flat no', 'flat number', 'flat #', 'flat', 'apartment', 'building']

streets = ["Main Street", "Market Road", "Gulshan Avenue", "Jinnah Boulevard", "Civil Lines", "Saddar Bazaar"]
localities = ["Gulberg", "Defence", "Model Town", "Clifton", "Nazimabad", "PECHS"]
cities = ["Karachi", "Lahore", "Islamabad", "Rawalpindi", "Faisalabad"]
provinces = ["Sindh", "Punjab", "Khyber Pakhtunkhwa", "Balochistan", "Gilgit-Baltistan", "Azad Kashmir"]

dataset = []

for _ in range(100):
    is_house = random.choice([True, False])
    
    street = random.choice(streets)
    locality = random.choice(localities)
    city = random.choice(cities)
    province = random.choice(provinces)
    
    if is_house:
        house_number = str(random.randint(1, 999))
        address_type = random.choice(house_keywords)
        address = f"{address_type} {house_number}, {street}, {locality}, {city}, {province}"
    else:
        flat_number = str(random.randint(101, 999))
        address_type = random.choice(apartment_keywords)
        address = f"{address_type} {flat_number}, {street}, {locality}, {city}, {province}"
    
    dataset.append(address)

# Save the dataset to a CSV file
filename = "random_addresses.txt"
with open(filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Address"])
    writer.writerows([address] for address in dataset)

print(f"Dataset saved to {filename}")


Dataset saved to random_addresses.txt.
